# BOPRA data preparation

Simo Mäkelä  
Helsinki Metropolia University of Applied Sciences  
1.10.2020

In [17]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from utils import zoll
from utils import nirs
from utils import misc
from utils.autoscale import autoscale

In [18]:
# parameters
case_id = 1

In [19]:
case_str = str(case_id).zfill(5)
zoll_filename = 'zoll' + case_str + '.json'
nirs_filename = 'nirs' + case_str + '.csv'
nirs_amend = 'nirs' + case_str + '_a2.csv'
json = zoll.LoadJSON("data/" + zoll_filename)

In [20]:
# load pulse waveform from channel 1
bp_df, bp_samplerate = zoll.LoadWaveform(json, hide_starttime=True, discard_by_status=False, channel=1)

Channel 1: CO2 kPa, Waveform
WaveType:  48
Duration:         5320.32 s
Total samples:     665040      Discarded:       0
Sample rate:          125 1/s  Sample time:  8000 μs
Min:                 -160      Max:           544



In [21]:
# for MAP
def ComputeAverage(waveform, window, samplerate, win_type=None):
    win = window * samplerate
    return waveform.rolling(win, win_type=win_type).mean()

bp_df.rename(columns={'BadSignal': 'Bad_MAP_auto'}, inplace=True)

In [22]:
# compute MAP using a 10 second sliding average
bp_df['MAP'] = ComputeAverage(bp_df['Waveform'], 10, bp_samplerate)

In [23]:
# merge with NIRS (rSO2) data
nirs_df = nirs.LoadCSV("data/"+nirs_filename, amendments="data/"+nirs_amend, discard_by_status=False)

df = pd.merge_asof(nirs_df[['rSO2', 'Mark', 'Bad_rSO2_auto', 'Bad_rSO2_manual']], bp_df[['MAP', 'Bad_MAP_auto']], on='Time', direction='nearest')

misc.NormalizeTime(df)
t_start, t_end = misc.CreateElapsedTimeColumn(df) # for colormap

df['T'] = df['Time'].apply(lambda x: x.strftime('%H:%M:%S'))
df['Time'] = df['T']
df.set_index('Time', inplace=True)


KeyError: &#39;HuonoSignaali2&#39;

In [16]:
df[['MAP', 'Bad_MAP_auto', 'rSO2', 'Mark', 'Bad_rSO2_auto', 'Bad_rSO2_manual', 'Elapsed']].to_csv('data/' + case_str + '.csv', na_rep='-')